In [67]:
from urllib.request import Request, urlopen
from urllib.error import URLError
from bs4 import BeautifulSoup, Comment
import re
import csv

url = "https://www.gatechdining.com/images/WeeklyMenuFC4-3_tcm251-34399.htm"
req = Request(url)

try:
    response = urlopen(req)
except URLError as e:
    if hasattr(e, 'reason'):
        print('Failed to reach a server.')
        print('Reason: ', e.reason)
    elif hasattr(e, 'code'):
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
    else:
        pass       
    
rd = response.read()

In [68]:
soup = BeautifulSoup(rd,'lxml')
week_id = soup.find(string=re.compile("Week of"))
print(week_id)
restaurant_name = soup.find(string=re.compile("RESTAURANT"))
if restaurant_name == None:
    print("Error: fail to achieve restraurant name!")
else:
    print(restaurant_name)

db = soup.find(string=re.compile("aData=new Object\(\)"))

Week of Monday April 3, 2017
GEORGIA TECH-FOOD COURT - RESTAURANT


In [69]:
db = db.replace('aData[', '').replace(']=new Array(', ',')

In [70]:
nutr_db = db.split(';')
nutr_db=nutr_db[1:len(nutr_db)-1]
nutr_db = [item.replace("'", "").replace("\r\n", "").split(',') for item in nutr_db]

In [71]:
# nutr_db

In [72]:
nutr_dict = {}
for item in nutr_db:
    item[-1]=item[-1].replace(r')', "")
    nutr_dict[item[0]] = item[1:]

In [73]:
# nutr_dict

In [74]:
c_id = []
for course in soup.find_all('input', onmouseover="wschk(1);"):
    c_id.append(course.get('id'))

In [75]:
aDays = {"1":"Monday","2":"Tuesday","3":"Wednesday","4":"Thursday","5":"Friday","6":"Saturday","7":"Sunday"}
aMeals = {"B":"Breakfast","L":"Lunch","S":"Dinner","X":"Late Night"}

menu = []
i=0
for ci in c_id:
    lst = []
    lst.extend([ci, aDays[ci[1]], aMeals[ci[2]]])
    lst.extend(nutr_dict[ci[-16:]])
    menu.append(lst)

In [76]:
# ht = '''
# <table class="dayinner">
# <tr class="brk"><td colspan="3" class="mealname">BREAKFAST</td></tr><tr class="brk"><td class="station">&nbsp;Hot Cereal</td>
# <td class="menuitem">
#    <div class="menuitem">
#    <input type="checkbox" class="chk" hidefocus="true" id="S1B0000010000084926_48570" onmouseover="wschk(1);" onmouseout="wschk(0);" onclick="rptlist(this);" />
#    <span class="ul" onmouseover="ws(this);" onclick="nf('0000084926_48570');" onmouseout="pcls(this);">Blueberries & Cream Oatmeal</span><img class="icon" src="v.gif" alt="Vegetarian" /><img class="icon" src="m.gif" alt="Mindful Item" /></div></td>
#    <td class="price"></td></tr>
# <tr class="brk"><td class="station">&nbsp;</td>
# <td class="menuitem">
#    <div class="menuitem">
#    <input type="checkbox" class="chk" hidefocus="true" id="S1B0000020000048637_41502" onmouseover="wschk(1);" onmouseout="wschk(0);" onclick="rptlist(this);" />
#    <span class="ul" onmouseovefr="ws(this);" onclick="nf('0000048637_41502');" onmouseout="pcls(this);">Classic Grits</span><img class="icon" src="g.gif" alt="Vegan" /><img class="icon" src="m.gif" alt="Mindful Item" /></div></td>
#    <td class="price"></td></tr>
# '''

# sp = BeautifulSoup(ht,'lxml')
# for i in sp.find_all('input', onmouseover="wschk(1);"):
#     print(i)
#     p=i.input
#     print(p)

In [77]:
# for item in menu[:3]:
#     print(item)

In [78]:
headers = ["Course ID", "Day in Week", "Meal", "Serving Size", "Calories", "Calories from Fat", "Total Fat(g)", "% DV", 
           "Saturated Fat(g)", "% DV", "Trans Fat", "Cholesterol(mg)", "% DV", "Sodium(mg)", "% DV", 
           "Total Carbohydrates(g)", "% DV", "Dietary Fiber(g)", "% DV", "Sugars (g)", "Protein(g)", "Vitamin A(%DV)",
           "Vitamin C(%DV)", "Calcium(%DV)", "Iron(%DV)", "Course Name", "Description", "Special"]

menu = [item[:27]+[item[-1]] for item in menu]

In [79]:
import csv
fname = "FOOD COURT RESTAURANTS.csv"
with open(fname, 'w', newline='') as csvfile:
    wrt = csv.writer(csvfile, delimiter=';', dialect='excel')
    wrt.writerow(headers)
    wrt.writerows(menu)
csvfile.close()